In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
PATH = r'/home/dgasch/proyectos/GIAR/'
ALF=2016
ALI=2008

* Actividad​: Se considera que un estudiante tiene ​actividad en un año determinado si: tiene 
alguna materia con el campo “Tipo de aprobación” distinto de “libre” ó si rindió algún final de 
esa u otra materia. Notar que ​actividad es una característica siempre asociada a un año 
determinado ​XX​. 
* (ALF) Año Lectivo Final​: Año que se toma como punto de partida para analizar hacia atrás 
las historias disponibles hasta 2008. En nuestro caso, por ahora, ALF = 2016. 
* (ALI) Año lectivo inicial​: Año a partir de que contamos con datos, En nuestro caso, por 
ahora es ALI = 2008. 
* (AUA) Año de Última Actividad​: Último año en que el estudiante haya tenido alguna 
actividad​. 
* (AI) Año de Ingreso​: Año en el que el estudiante haya realizado su primera actividad. 
Alumno ​activo a la fecha ​XX​: Se considera que un estudiante está “activo a la fecha ​XX​” si 
en el año ​XX ​registró ​actividad​.  
* (AE) Año de egreso​: Año en que aprobó “Proyecto Final”.  
* Condición de Egresado​: La condición de ​egresado se puede asignar a un alumno en el 
año de egreso o posteriores. Ver que se necesita indicar un año para ver si pertenece a esa 
categoría. 
* Condición de Desertor​: Se considera que un alumno es desertor para el año ​XX si no 
registra actividad ni en el año ​XX ni en el año ​XX - 1​. Ver que se necesita indicar un año para 
ver si pertenece a esa categoría

## Cargo las bases de datos de alumnos y finales

In [2]:
alumnos=pd.read_excel(PATH+'Datos-Alumnos-SIGA.xlsx')
#Agrego la columna de desertor
alumnos['deserto']=0
finales=pd.read_excel(PATH+'Finales-Alumnos-SIGA.xlsx')
cursadas=pd.read_excel(r'/home/dgasch/proyectos/GIAR/Cursadas-Alumnos-SIGA.xlsx')

Selecciono los alumnos y el ano en que egresaron
* (AE) Año de egreso​: Año en que aprobó “Proyecto Final”.

In [3]:
#Genero un listado del ano de egreso
ano_egreso=finales[(finales['Materia']=='Proyecto Final') & (finales['Aprobado']==1)][['Codigo Alumno','Año']]

Si egresó se lo categoriza como “noDesertor” con las características calculadas 
hasta AUA-2 (en este caso el AUA coincide con AE) (siempre que ALI < AUA-2). Se 
pasa al siguiente individuo (vuelve al paso 1 para el individuo siguiente). 

In [4]:
# Genero una lista de los alumnos que egresaron en un periodo anterior a la fecha de chequeo
alumnos_egreso=ano_egreso[ano_egreso['Año']<=(ALF)]['Codigo Alumno']

Busco todos los alumnos y la maxima fecha de final entre los que no estan entre los egresados y todos aquellos que cumplan con la condicion 
Si no egresó y si AUA < (ALF-1), se lo categoriza como “desertor” con las 
características calculadas hasta AUA inclusive. Se pasa al siguiente individuo, va al 
paso 1.

In [5]:
# Filtro los alumnos de acuerdo al ultimo ano que rindieron finales
alumno_finales_ult=finales[finales['Año']<=(ALF)].groupby('Codigo Alumno' ,as_index = False)['Año'].max()
# Genero una lista con los alumnos que rindieron finales
alumnos_rindieron_finales=alumno_finales_ult[(alumno_finales_ult['Año']<(ALF-1)) & (~alumno_finales_ult['Codigo Alumno'].isin(alumnos_egreso))]['Codigo Alumno']

Llegado este punto sólo puede darse la situación de que su última actividad esté 
dentro de los últimos dos años, ALF y ALF-1. Hay dos alternativas para estos casos: 
Alternativa 1  
Si AI > (ALF-2) se lo descarta y se pasa al siguiente individuo, paso 1; 
(no podemos asegurar el resultado) (esto en rigor es parte del 
preprocesamiento y limpieza inicial) 
Sino se lo categoriza como “noDesertor” con las características 
calculadas hasta (ALF-2). 
Alternativa 2 
Se lo descarta y se pasa al siguiente individuo, paso 1. (habrá que ver 
cuántos son para considerar descartarlos o no. lo que sea más facil) 


In [7]:
alumno_cursadas_ult=cursadas[cursadas['Ciclo Lectivo de Cursada']<=(ALF)].groupby('Codigo Alumno' ,as_index = False)['Ciclo Lectivo de Cursada'].max().reset_index()

In [8]:
alumnos_cursaron=alumno_cursadas_ult[(alumno_cursadas_ult['Ciclo Lectivo de Cursada']<(ALF-1)) & (~alumno_cursadas_ult['Codigo Alumno'].isin(alumnos_egreso))]['Codigo Alumno']

In [9]:
alumnos_cursaron

32        204220
34        459546
36        785181
37        806880
38        865092
39        902073
40        936023
43       1028789
44       1046032
46       1086261
48       1088919
52       1096898
53       1099723
55       1102849
61       1116964
62       1118924
63       1120074
67       1125760
68       1126763
70       1128759
71       1138182
72       1141028
73       1144480
75       1146932
76       1147626
77       1149106
79       1153730
80       1154151
81       1154783
83       1155982
          ...   
6126    10001775
6127    10001780
6132    10001787
6146    10001828
6147    10001830
6153    10001846
6164    10001871
6169    10001886
6171    10001888
6181    10001911
6185    10001921
6189    10001926
6199    10001941
6204    10001947
6212    10001961
6213    10001963
6221    10001977
6223    10001985
6226    10001991
6228    10001993
6233    10002005
6235    10002013
6239    10002017
6240    10002018
6245    10002028
6252    10002066
6259    10002558
6268    100033

In [11]:
alumnos_desertores=pd.concat([alumnos_cursaron,alumnos_rindieron_finales])

In [12]:
alumnos['deserto'][alumnos['Codigo Alumno'].isin(alumnos_desertores)]=1

/home/dgasch/maestria/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
alumnos.deserto.value_counts()

0    5266
1    3069
Name: deserto, dtype: int64

In [16]:
#Guardo el listado de alumnos
alumnos.to_csv(r'/home/dgasch/proyectos/GIAR/alumnos_desertores.csv',sep=';')